In [3]:
import pandas as pd 
dataset = pd.read_csv("Market_Basket_Analysis_v1.csv")
dataset

,Order Date,Order ID,Customer ID,Ship Mode,Description,Information,Customer Name,Segment,City,State,Country,Market,Region,Attribute,Item Name,Quantity,Product ID,Unit Price,Total Sales
0,31-01-2021 00:00,BU-2013-7380,MH-7455,Standard Class,BU-2013-7380MH-7455Mark Hamilton,MH-7455Mark HamiltonConsumerBurgasBurgas,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,Item name1,shrimp,167,10002439.0,74.69,12473.23
1,31-01-2021 00:00,BU-2013-7380,MH-7455,Standard Class,BU-2013-7380MH-7455Mark Hamilton,MH-7455Mark HamiltonConsumerBurgasBurgas,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,Item name2,almonds,278,10000353.0,15.28,4247.84
2,31-01-2021 00:00,BU-2013-7380,MH-7455,Standard Class,BU-2013-7380MH-7455Mark Hamilton,MH-7455Mark HamiltonConsumerBurgasBurgas,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,Item name3,avocado,324,10000994.0,46.33,15010.92
3,31-01-2021 00:00,BU-2013-7380,MH-7455,Standard Class,BU-2013-7380MH-7455Mark Hamilton,MH-7455Mark HamiltonConsumerBurgasBurgas,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,Item name4,vegetables mix,124,10000475.0,58.22,7219.28
4,31-01-2021 00:00,BU-2013-7380,MH-7455,Standard Class,BU-2013-7380MH-7455Mark Hamilton,MH-7455Mark HamiltonConsumerBurgasBurgas,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,Item name5,green grapes,680,10003338.0,86.31,58690.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72624,30-03-2023 00:00,US-2013-151225,GK-14620,Standard Class,US-2013-151225GK-14620Grace Kelly,GK-14620Grace KellyCorporateLimaLima (city),Grace Kelly,Corporate,Lima,Lima (city),Peru,LATAM,South,Item name1,chicken,290,10004883.0,86.72,25148.80
72625,30-03-2023 00:00,US-2013-151225,GK-14620,Standard Class,US-2013-151225GK-14620Grace Kelly,GK-14620Grace KellyCorporateLimaLima (city),Grace Kelly,Corporate,Lima,Lima (city),Peru,LATAM,South,Item name2,tropical fruit,314,10001990.0,56.47,17731.58
72626,30-03-2023 00:00,US-2013-151225,GK-14620,Standard Class,US-2013-151225GK-14620Grace Kelly,GK-14620Grace KellyCorporateLimaLima (city),Grace Kelly,Corporate,Lima,Lima (city),Peru,LATAM,South,Item name3,other vegetables,447,10000425.0,92.13,41182.11
72627,30-03-2023 00:00,US-2013-151225,GK-14620,Standard Class,US-2013-151225GK-14620Grace Kelly,GK-14620Grace KellyCorporateLimaLima (city),Grace Kelly,Corporate,Lima,Lima (city),Peru,LATAM,South,Item name4,vinegar,701,10001410.0,89.69,62872.69
